In [1]:
import torch
import torchvision
from torchvision import transforms, datasets
import numpy as np

## 3 аттрибута тензора в pytorch:

dtype - тип данных хранимых в тензоре (float32, etc.)

device - на чём обрабатывается данный тензор (тензоры которые взаимодействую должны быть на одном проце/видяхе)

layout - как данные расположены в памяти (не трогай это, дефолтная настройка норм)

# Создание тензоров (прямо как в NumPy):

In [2]:
# единичный тензор
print(torch.eye(3))

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


In [3]:
# нулевой
torch.zeros(2,2)

tensor([[0., 0.],
        [0., 0.]])

In [4]:
# единичный
torch.ones(2,2)

tensor([[1., 1.],
        [1., 1.]])

# Создание тензоров из данных:

Factory function has more tweaking parameters than class constructor, so we will use them more often. They also infere the dtype (they get data in i.e. int32, they save it as int32).

Beware that torch.Tensor and torch.tensor CREATE a copy of input data in memory, when torch.as_tensor and torch.from_numpy just mirror the existing data (so the last is more memory efficient, but if the original data is changed, tensor changed too. Data MUST be in NumPy array).

For casual use - torch.tensor()

For speed boost - torch.as_tensor()

In [5]:
data = np.array([1,2,3])

t1 = torch.Tensor(data) # class constructor
t2 = torch.tensor(data) # factory function
t3 = torch.as_tensor(data) # factory function
t4 = torch.from_numpy(data) # factory function

# Reshaping the the tensors, squeezing

.reshape() or .view()(одно и то же, просто разные названия) - прямое изменение размеров тензора

.squeeze() - удаление всех осей с длинной 1.

.unsqueeze() - добавляет ось с длиной 1, это позволяет изменять ранг тензора.



In [6]:
t = torch.tensor([
    [1,1,1,1],
    [2,2,2,2],
    [3,3,3,3]
], dtype=torch.float32)
t.shape

torch.Size([3, 4])

In [7]:
# number of elements check
print(torch.tensor(t.shape).prod())
print(t.numel())

tensor(12)
12


In [8]:
print(t.reshape(2, 1, 2, 3)) # reshape to rank-4 tensor (from rank-2)
print(t.reshape(2, 6))

tensor([[[[1., 1., 1.],
          [1., 2., 2.]]],


        [[[2., 2., 3.],
          [3., 3., 3.]]]])
tensor([[1., 1., 1., 1., 2., 2.],
        [2., 2., 3., 3., 3., 3.]])


In [9]:
print(t.reshape(1,12)) # tensor rank 2
print(t.reshape(1,12).squeeze()) # tensor rank 1

# маленький трюк - если не знаем сколько компонентов будет в тензоре который мы хотим
# "вытянуть" сохранив второй ранг мы пишем "1, -1" во втором атрибуте .reshape
print(t.reshape(1,-1))

# если хотим перевести его в тензор первого ранга пишем "-1"
print(t.reshape(-1))

tensor([[1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.]])
tensor([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.])
tensor([[1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.]])
tensor([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.])


# Flattening, Tensor size for neural network

.flatten() - позволяет зарешейпить тензор любого ранга в тензор первого ранга

Однако для нейросети абсолютно "плоские" данные нам не нужны, т.к. она должна различать batch (какие данные к какой введённой переменой относятся).

Нейронка работает с тензорами ранга 4, вот что каждая из осей представляет(на примере тензора ниже):

Batch - contains 3 images

Image - contains 1 color channel (cause grayscale)

Color channel - contains 4 arrays(rows/hight)

4 arrays(rows/hight) - contains 4 pixel values (columns/width)

In [10]:
# create 3 "images" 4x4 pixels, grayscale

t1 = torch.tensor([
    [1,1,1,1],
    [1,1,1,1],
    [1,1,1,1],
    [1,1,1,1]
])
t2 = torch.tensor([
    [2,2,2,2],
    [2,2,2,2],
    [2,2,2,2],
    [2,2,2,2]
])
t3 = torch.tensor([
    [3,3,3,3],
    [3,3,3,3],
    [3,3,3,3],
    [3,3,3,3]
])

# stack them to make a tensor with rank 3 (new axis represents batch), now we have a 3
# "grayscale" pictures in the resulting tensor
t = torch.stack((t1, t2, t3))
t.shape

torch.Size([3, 4, 4])

In [11]:
# and a new axis to this tensor to create a "batch" axis for discening the "pictures"
t = t.reshape(3, 1, 4, 4)
t

tensor([[[[1, 1, 1, 1],
          [1, 1, 1, 1],
          [1, 1, 1, 1],
          [1, 1, 1, 1]]],


        [[[2, 2, 2, 2],
          [2, 2, 2, 2],
          [2, 2, 2, 2],
          [2, 2, 2, 2]]],


        [[[3, 3, 3, 3],
          [3, 3, 3, 3],
          [3, 3, 3, 3],
          [3, 3, 3, 3]]]])

## how to use flatten

We can't flatten the whole tensor, cause it will become a simple vector and we won't be able to know where are the "start" and the "end" of each "picture". So we flatten this tensor in such a way that wa preserve the "batch" axis (flattening the color channel with hight and width axes), which tells us which "picture" is which.

In [12]:
# bad idea - got vector with 48 pixel, no way to know from which picture each pixel comes from
a = t.flatten()
print('bad  - ', a.shape)

# good idea - got 3 images with 16 pixels
t = t.flatten(start_dim=1) # start flattening from axis=1 (it`s an index)
print('good - ', t.shape)

bad  -  torch.Size([48])
good -  torch.Size([3, 16])


# Broadcasting for element-wise operations

Broadcasting is transforming a lower rank tensor to match the shape of the tensor with which we want it to perform an element-wise operation.

All element-wise operations works with tensors due to the broadcasting.

In [13]:
print(t.eq(1)) # equal 1
print(t.abs()) # взятие модуля
print(t + 3)
print(t.neg())

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]])
tensor([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
        [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
        [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]])
tensor([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2],
        [-3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3]])


# Reduction operation - .ArgMax()

Reduction operation on tensor is an operation that reduced the number of elements contained within the tensor.

We can use .mean(),

.argmax() returns the index of the max value inside the tensor

In [57]:
t4 = torch.tensor([
    [0,1,0],
    [2,0,2],
    [0,3,0]
], dtype=torch.float32)
print(t4.shape)

print(t4.sum()) # sum all elements = reduce this tensor to one axis
print(t4.sum(dim=0))
print(t4.sum(dim=0).shape) # sum along axis 1 (sum columns) = reduce this tensor to one axis

torch.Size([3, 3])
tensor(8.)
tensor([2., 4., 2.])
torch.Size([3])


In [58]:
print('value   = ', t4.max())     # MAX value 
print('flatten = ', t4.flatten()) # flatten tensor (count indices from '0')
print('index   = ', t4.argmax())  # index of the MAX value (yep, '3' has index=7)


value   =  tensor(3.)
flatten =  tensor([0., 1., 0., 2., 0., 2., 0., 3., 0.])
index   =  tensor(7)


In [59]:
print(t4.max(dim=0), '\n') # max values on axis=0
print(t4.max(dim=1)) # first tensor - max values, second tensor - their indices

torch.return_types.max(
values=tensor([2., 3., 2.]),
indices=tensor([1, 2, 1])) 

torch.return_types.max(
values=tensor([1., 2., 3.]),
indices=tensor([1, 2, 1]))


In [66]:
# if we wanna get the results not as tensors, but as ints/floats/np.arrays:

print(t4.mean())             # got tensor
print(t4.mean().item(), '\n') # got float32

print(t4.mean(dim=0))                # got tensor
print(t4.mean(dim=0).tolist(), '\n') # got np.array of float32

tensor(0.8889)
0.8888888955116272 

tensor([0.6667, 1.3333, 0.6667])
[0.6666666865348816, 1.3333333730697632, 0.6666666865348816] 

